In [ ]:
from spider_lianjia.spiders.chengjiao import get_chengjiao_one
from spider_lianjia.spiders.zaishou import get_zaishou_one
g1=get_chengjiao_one().get_chengjiao('悦泰福里')
g2=get_zaishou_one().get_zaishou('悦泰福里')

In [ ]:
from pandas import DataFrame,Series
import pandas as pd
import re
df_c=DataFrame(g1,columns=['title','totalPrice','unitPrice','room','type','area','quxian','district','builtYear','guapaiPrice','dealCycle','dealDate'])
df_c['area']=df_c['area'].apply(lambda x:float(re.search('\d+(\.\d+)?',x).group()))
# df_c['totalPrice']=df_c['totalPrice'].apply()
df_c=df_c.astype({'totalPrice':'float','unitPrice':'float','builtYear':'int','guapaiPrice':'float','dealCycle':'int'})
df_c['dealDate']=pd.to_datetime(df_c['dealDate'])

In [ ]:
df_z=DataFrame(g2,columns=['title','totalPrice','unitPrice','room','type','area','quxian','district','seller','builtYear','guapaiDate','preDate'])
df_z['builtYear']=df_z['builtYear'].map(lambda x:int(re.search('\d+',x).group()))
df_z['area']=df_z['area'].apply(lambda x:float(re.search('\d+(\.\d+)?',x).group()))
df_z=df_z.astype({'totalPrice':'float','unitPrice':'float'})
df_z['guapaiDate']=df_z['guapaiDate'].apply(lambda x:str.replace(x,'-','.'))#要把2021-01-10转换为2021.01.10
df_z['guapaiDate']=pd.to_datetime(df_z['guapaiDate'])
df_z['preDate']=df_z['preDate'].apply(lambda x:str.replace(x,'-','.'))
df_z['preDate'].apply(lambda x:pd.to_datetime(x) if x!='暂无数据' else 0)
df_z.dtypes

In [25]:
d=round(df_z.groupby(['title'])['unitPrice'].describe(),0)
d.columns.name='正在售'
d

正在售,count,mean,std,min,25%,50%,75%,max
title,,,,,,,,
悦泰福里,23.0,27370.0,6379.0,19641.0,22848.0,24571.0,33038.0,42036.0


In [23]:
d=round(df_c.groupby(['title'])['unitPrice'].describe(),0)
d.columns.name='已成交'
d

已成交,count,mean,std,min,25%,50%,75%,max
title,,,,,,,,
悦泰福里,25.0,20940.0,4693.0,11554.0,18858.0,20228.0,23581.0,30314.0


In [26]:
df_c

,title,totalPrice,unitPrice,room,type,area,quxian,district,builtYear,guapaiPrice,dealCycle,dealDate
0,悦泰福里,238.0,25336.0,2室2厅,南 北,93.94,西岗,大菜市,2014,243.0,367,2021-05-30
1,悦泰福里,135.0,19256.0,2室1厅,南,70.11,西岗,大菜市,2014,138.0,163,2021-05-17
2,悦泰福里,214.0,22760.0,2室2厅,南 北,93.94,西岗,大菜市,2014,228.0,23,2021-05-16
3,悦泰福里,202.0,21796.0,2室2厅,南 北,92.45,西岗,大菜市,2014,209.0,37,2021-04-23
4,悦泰福里,223.0,24122.0,2室1厅,南 北,92.45,西岗,大菜市,2013,226.0,30,2021-03-28
5,悦泰福里,218.0,23581.0,2室1厅,南 北,92.45,西岗,大菜市,2015,220.0,3,2021-03-14
6,悦泰福里,139.0,19826.0,1室1厅,南,70.11,西岗,大菜市,2015,142.0,265,2021-03-10
7,悦泰福里,136.0,19399.0,1室1厅,南,70.11,西岗,大菜市,2014,140.0,6,2021-03-08
8,悦泰福里,133.0,18971.0,1室1厅,南,70.11,西岗,大菜市,2014,137.0,64,2021-02-28
9,悦泰福里,133.0,18971.0,1室1厅,南,70.11,西岗,大菜市,2014,143.5,17,2020-12-09
